In [1]:
let x = ref 0;;

val x : int ref = {contents = 0}


In [2]:
!x;;

- : int = 0


In [3]:
x := !x + 1

- : unit = ()


In [4]:
!x;;

- : int = 1


In [5]:
let x = ref 42;;
let y = ref 42;;
let z = x;;
x := 43;;
let w = !y + !z;;

val x : int ref = {contents = 42}


val y : int ref = {contents = 42}


val z : int ref = {contents = 42}


- : unit = ()


val w : int = 85


In [6]:
let counter = ref 0

let next_val =
  fun () ->
    counter := !counter + 1;
    !counter

val counter : int ref = {contents = 0}


val next_val : unit -> int = <fun>


In [7]:
next_val ();;

next_val ();;

- : int = 1


- : int = 2


In [8]:
let next_val_broken = fun () ->
  let counter = ref 0 in
  incr counter;
  !counter


val next_val_broken : unit -> int = <fun>


In [9]:
next_val_broken ();;
next_val_broken ();;

- : int = 1


- : int = 1


In [10]:
module Counter = struct
  module Make() = struct
    type t = int ref
    let counter = ref 0
    let next_val () =
      counter := !counter + 1;
      !counter
  end
end

module Counter :
  sig
    module Make :
      functor () ->
        sig
          type t = int ref
          val counter : int ref
          val next_val : unit -> int
        end
  end


In [11]:
module Counter1 = Counter.Make();;
Counter1.next_val ();;
Counter1.next_val ();;
module Counter2 = Counter.Make();;
Counter2.next_val ();;


module Counter1 :
  sig type t = int ref val counter : int ref val next_val : unit -> int end


- : int = 1


- : int = 2


module Counter2 :
  sig type t = int ref val counter : int ref val next_val : unit -> int end


- : int = 1


In [12]:
let rec fact_rec n = if n = 0 then 1 else n * fact_rec (n - 1)

let fact0 = ref (fun x -> x + 0)

let fact n = if n = 0 then 1 else n * !fact0 (n - 1);;
fact0 := fact

let _ = fact 5
  

val fact_rec : int -> int = <fun>


val fact0 : (int -> int) ref = {contents = <fun>}


val fact : int -> int = <fun>


- : unit = ()


- : int = 120


In [13]:
let r1 = ref 42
let r2 = ref 42

let _ = assert (not (r1 == r2))
let _ = assert (r1 != r2)
let _ = assert (!r1 == !r2)
let _ = assert (r1 = r2)

val r1 : int ref = {contents = 42}


val r2 : int ref = {contents = 42}


- : unit = ()


- : unit = ()


- : unit = ()


- : unit = ()


In [2]:
let t = Hashtbl.create 16;; 
List.fold_left (fun () x -> Hashtbl.add t x (string_of_int x)) () (List.init 16 (fun x -> x));;
let _ = Hashtbl.mem t 5;;

val t : ('_weak1, '_weak2) Hashtbl.t = <abstr>


- : unit = ()


- : bool = true


In [5]:
module PairHash = struct
  type t = string * int
  let equal (a1, b1) (a2, b2) = a1 = a2 && b1 = b2
  let hash (a, b) = Hashtbl.hash (a, b)
end

module PairHashtbl = Hashtbl.Make(PairHash)

module PairHash :
  sig
    type t = string * int
    val equal : 'a * 'b -> 'a * 'b -> bool
    val hash : 'a * 'b -> int
  end


module PairHashtbl :
  sig
    type key = PairHash.t
    type 'a t = 'a Hashtbl.Make(PairHash).t
    val create : int -> 'a t
    val clear : 'a t -> unit
    val reset : 'a t -> unit
    val copy : 'a t -> 'a t
    val add : 'a t -> key -> 'a -> unit
    val remove : 'a t -> key -> unit
    val find : 'a t -> key -> 'a
    val find_opt : 'a t -> key -> 'a option
    val find_all : 'a t -> key -> 'a list
    val replace : 'a t -> key -> 'a -> unit
    val mem : 'a t -> key -> bool
    val iter : (key -> 'a -> unit) -> 'a t -> unit
    val filter_map_inplace : (key -> 'a -> 'a option) -> 'a t -> unit
    val fold : (key -> 'a -> 'b -> 'b) -> 'a t -> 'b -> 'b
    val length : 'a t -> int
    val stats : 'a t -> Hashtbl.statistics
    val to_seq : 'a t -> (key * 'a) Seq.t
    val to_seq_keys : 'a t -> key Seq.t
    val to_seq_values : 'a t -> 'a Seq.t
    val add_seq : 'a t -> (key * 'a) Seq.t -> unit
    val replace_seq : 'a t -> (key * 'a) Seq.t -> unit
    val of_seq : (key * 'a) Seq.t

In [7]:
let rec fib n = if n < 2 then 1 else fib (n - 1) + fib (n - 2)

let _ = fib 45

val fib : int -> int = <fun>


- : int = 1836311903


In [10]:
let fib_memo n =
  let memo = Hashtbl.create 16 in
  let rec fib_memo' n =
    if n < 2 then 1
    else
      match Hashtbl.find_opt memo n with
      | Some v -> v
      | None ->
        let v = fib_memo' (n - 1) + fib_memo' (n - 2) in
        Hashtbl.add memo n v;
        v
  in
  fib_memo' n

let _ = fib_memo 45


val fib_memo : int -> int = <fun>


- : int = 1836311903


In [25]:
let fib_0 = ref (fun x -> x)
let rec fib_rec n = if n < 2 then 1 else (!fib_0) (n - 1) + (!fib_0) (n - 2)
let _ = fib_0 := fib_rec

let _ = fib_rec 45


val fib_0 : ('_weak24 -> '_weak24) ref = {contents = <fun>}


val fib_rec : int -> int = <fun>


- : unit = ()


- : int = 1836311903


In [26]:
let fib_hash = Hashtbl.create 16
let fib_mem = 
  fun n ->
    match Hashtbl.find_opt fib_hash n with
    | Some v -> v
    | None -> let v = fib_rec n in
      Hashtbl.add fib_hash n v;
      v
let _ = fib_0 := fib_mem
let _ = fib_rec 45



val fib_hash : ('_weak25, '_weak26) Hashtbl.t = <abstr>


val fib_mem : int -> int = <fun>


- : unit = ()


- : int = 1836311903


In [28]:
let memo_rec f =
  let h = Hashtbl.create 16 in
  let rec g x =
    try Hashtbl.find h x
    with Not_found ->
      let y = f g x in
      Hashtbl.add h x y;
      y
  in
  g

let fib_memo =
  let fib self n =
    if n < 2 then 1 else self (n - 1) + self (n - 2)
  in
  memo_rec fib

let _ = fib_memo 45


val memo_rec : (('a -> 'b) -> 'a -> 'b) -> 'a -> 'b = <fun>


val fib_memo : int -> int = <fun>


- : int = 1836311903
